# Create watersheds and catchments

AKSSF project has ~ 500 sites that have been shifted to the flow networks.
We need to shift these sites to the stream raster, extract the appropriate
catchments and calculate watersheds for each. This will make extracting spatial
and climatic covariates for modeling go much faster.


In [20]:
# merge flow accumulation grids for Cook Inlet using mosaic to new raster

import arcpy
import os

arcpy.env.workspace = r"T:\Aquatic\AKSSF\NHDPlus\CookInlet_20201216"
flowacc_rasters = []
raster_folders = arcpy.ListWorkspaces("*Rasters*")
print(raster_folders)

for folder in raster_folders:
    arcpy.env.workspace = folder
    rasters = arcpy.ListRasters()
    for raster in rasters:
        if raster == 'fac.tif':
            flowacc_raspath = os.path.normpath(os.path.join(folder, raster))
            flowacc_rasters.append(flowacc_raspath)

print(flowacc_rasters)
print(flowacc_rasters[1])
dataset = flowacc_rasters[1]
spatial_ref = arcpy.Describe(dataset).spatialReference

arcpy.MosaicToNewRaster_management(flowacc_rasters, r"W:\GIS\AKSSF Southcentral\Temporary", "ci_fac_merge.tif", spatial_ref, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")


['T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020202', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020301', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020302', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020401', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020402', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020501', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020502', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020503', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020504', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020505', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020601', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020602', 'T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216\\HRNHDPlusRasters19020800'

<Result 'W:\\GIS\\AKSSF Southcentral\\Temporary\\ci_fac_merge.tif'>

In [22]:
# Combine point files for Bristol Bay and sites outside Bristol Bay so we have
# one file of points that have been shifted to the vector network.
from arcpy import management

bb_sites = r"T:\Aquatic\AKSSF\AKSSF_Hydrography.gdb\bb_md_verified_dm"
other_sites = r"T:\Aquatic\AKSSF\AKSSF_Hydrography.gdb\sites_outside_bb_verified_DM"

arcpy.management.Merge([bb_sites, other_sites], r"W:\GIS\AKSSF Southcentral\Temporary\all_sites")

<Result 'W:\\GIS\\AKSSF Southcentral\\Temporary\\all_sites.shp'>

In [29]:
# shift points to cell of highest flow accumulation
import os
import arcpy
from arcpy import env
from arcpy.sa import *

# Set environment settings
env.workspace = "C:/sapyexamples/data"

# Set local variables
inPourPoint = r"W:\GIS\AKSSF Southcentral\Temporary\all_sites.shp"
inFlowAccum = r"W:\GIS\AKSSF Southcentral\Temporary\ci_fac_merge.tif"
tolerance = 15

arcpy.management.AddField(inPourPoint, "numID", "LONG")

idField = 'numID'
rec = 0

def autoIncrement():
    global rec
    pStart    = 1
    pInterval = 1
    if (rec == 0):
        rec = pStart
    else:
        rec += pInterval
    return rec

rows = arcpy.UpdateCursor(inPourPoint)

for row in rows:
    row.setValue(idField, autoIncrement())
    rows.updateRow(row)

del row, rows

ExecuteError: ERROR 000875: Output raster: C:/sapyexamples/data\SnapPou_all_sit1.tif's workspace is an invalid output workspace.
ERROR 000581: Invalid parameters.
Failed to execute (SnapPourPoint).


In [4]:
import arcpy
inPourPoint = r"W:\GIS\AKSSF Southcentral\Temporary\all_sites.shp"
inFlowAccum = r"W:\GIS\AKSSF Southcentral\Temporary\ci_fac_merge.tif"
tolerance = 15
idField = 'numID'

# Execute SnapPourPoints
outSnapPour = arcpy.sa.SnapPourPoint(inPourPoint, inFlowAccum, tolerance,
                            idField)

# Save the output
outSnapPour.save(r"W:\GIS\AKSSF Southcentral\Temporary\sites_snap.tif")


Working on creating watersheds. So far, have shifted points to the cell of highest accumulation
and they look good in arcgis. Need to find a way to use a search cursor to cycle through the points
in the shifted file, select/intersect with the correct flow direction grid, and input the point
file with the selected point to the watershed tool.

Found some code that showed out to use a search cursor to select a feature in a fc and input to
a geoprocessing tool (will only act on the selection). But, still need a way to know which grid
is needed for each point's watershed. For NHDPlus, can use HU8 code, for synthetic networks, may
want to create polygon boundaries of the grids and intersect with points so there is an attribute
that indicates which grid to use (flow direction grid).

In [6]:
import arcpy
import os

fc = r"W:\GIS\AKSSF Southcentral\Temporary\sites_snap"
huc8 = r"S:\Leslie\GIS\NHD Harmonized\WBD_19_GDB.gdb\WBDHU8"
outfc = r"W:\GIS\AKSSF Southcentral\Temporary\sites_snap_huc8"
arcpy.analysis.SpatialJoin(fc, huc8, outfc)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Target Features: Dataset W:\GIS\AKSSF Southcentral\Temporary\sites_snap does not exist or is not supported
Failed to execute (SpatialJoin).


In [ ]:
arcpy.MakeFeatureLayer_management(fc, 'layer')

with arcpy.da.SearchCursor('layer', ['numID']) as cursor:
    for row in cursor:
        arcpy.SelectLayerByAttribute_management('layer', 'NEW_SELECTION', '{}={}'.format('numID', Value))
        # generate the minimum bounding geometry for this layer
        arcpy.MinimumBoundingGeometry_management('layer','in_memory\\temp','ENVELOPE')

    # use the extent of the bounding geometry as the geoprocessing extent
    desc = arcpy.Describe('in_memory\\temp')
    arcpy.env.extent = desc.extent

    arcpy.sa.Watershed('layer', <output>)

    arcpy.Delete_management('in_memory\\temp') # Clean up
    arcpy.env.extent = 'MAXOF' # reset to 'big' so the next MBG isn't affected.